In [2]:
! pip install beakerx
! pip install cimcb_lite

     |████████████████████████████████| 18.1 MB 82 kB/s 
     |████████████████████████████████| 212.4 MB 58 kB/s 
     |████████████████████████████████| 89 kB 8.2 MB/s 
     |████████████████████████████████| 198 kB 62.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=7c073aad9d341a774182fdb3b5186a4edc33fdc03468786478b3563d16bccb03
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark
     |████████████████████████████████| 47 kB 1.9 MB/s 


[link](https://cimcb.github.io/MetabWorkflowTutorial/)

In [3]:
import numpy as np
import pandas as pd

from beakerx.object import beakerx
from sklearn.model_selection import train_test_split

import cimcb_lite as cb

beakerx.pandas_display_table()  # by default display pandas tables as BeakerX interactive tables

print('All packages successfully loaded')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


BeakerxHTML(value='You need beakerx_tabledisplay to use this')

All packages successfully loaded


The study used in this tutorial has been previously published as an open access article Chan et al. (2016), in the British Journal of Cancer, and the deconvolved and annotated data file deposited at the Metabolomics Workbench data repository (Project ID PR000699). The data can be accessed directly via its project DOI:10.21228/M8B10B 1H-NMR spectra were acquired at Canada’s National High Field Nuclear Magnetic Resonance Centre (NANUC) using a 600 MHz Varian Inova spectrometer. Spectral deconvolution and metabolite annotation was performed using the Chenomx NMR Suite v7.6. Unfortunately, the Raw NMR data is unavailable.

In [5]:
! wget https://www.metabolomicsworkbench.org/studydownload/ST001047.zip
! gzip *zip

--2021-09-02 06:30:46--  https://www.metabolomicsworkbench.org/studydownload/ST001047.zip
Resolving www.metabolomicsworkbench.org (www.metabolomicsworkbench.org)... 132.249.223.7
Connecting to www.metabolomicsworkbench.org (www.metabolomicsworkbench.org)|132.249.223.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146347 (143K) [application/zip]
Saving to: ‘ST001047.zip’

ST001047.zip        100%[===================>] 142.92K   368KB/s    in 0.4s    

2021-09-02 06:30:48 (368 KB/s) - ‘ST001047.zip’ saved [146347/146347]



In [6]:
! gunzip ST001047.zip.gz
! unzip ST001047.zip
! ls

Archive:  ST001047.zip
   creating: ST001047/
  inflating: ST001047/Gastric_NMR.zip  
'DRCCStudySummary.php?Mode=SetupRawDataDownload'   ST001047
 sample_data					   ST001047.zip


In [7]:
! wget https://cimcb.github.io/MetabWorkflowTutorial/GastricCancer_NMR.xlsx

--2021-09-02 06:31:51--  https://cimcb.github.io/MetabWorkflowTutorial/GastricCancer_NMR.xlsx
Resolving cimcb.github.io (cimcb.github.io)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to cimcb.github.io (cimcb.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154870 (151K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘GastricCancer_NMR.xlsx’

GastricCancer_NMR.x 100%[===================>] 151.24K  --.-KB/s    in 0.03s   

2021-09-02 06:31:51 (5.23 MB/s) - ‘GastricCancer_NMR.xlsx’ saved [154870/154870]



# 2. Load Data and Peak sheet
This workflow requires data to be uploaded as a Microsoft Excel file, using the Tidy Data framework (i.e. each column is a variable, and row is an observation). As such, the Excel file should contain a Data Sheet and Peak Sheet. The Data Sheet contains all the metabolite concentrations and metadata associated with each observation (requiring the inclusion of the columns: Idx, SampleID, and Class). The Peak Sheet contains all the metadata pertaining to each measured metabolite (requiring the inclusion of the columns: Idx, Name, and Label). Please inspect the Excel file used in this tutorial before proceeding.
The code cell below loads the Data and Peak sheets from an Excel file, using the CIMCB helper function load_dataXL(). When this is complete, you should see confirmation that Peak (stored in the Peak worksheet in the Excel file) and Data (stored in the Data worksheet in the Excel file) tables have been loaded:

In [8]:
filename = 'GastricCancer_NMR.xlsx'

# load peak and data tables into 2 variables
dataTable, peakTable = cb.utils.load_dataXL(filename, DataSheet='Data', PeakSheet='Peak')

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 140 TOTAL PEAKS: 149
Done!


In [9]:
display(dataTable)

,Idx,SampleID,SampleType,Class,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12,M13,M14,M15,M16,M17,M18,M19,M20,M21,M22,M23,M24,M25,M26,M27,M28,M29,M30,M31,M32,M33,M34,M35,M36,...,M110,M111,M112,M113,M114,M115,M116,M117,M118,M119,M120,M121,M122,M123,M124,M125,M126,M127,M128,M129,M130,M131,M132,M133,M134,M135,M136,M137,M138,M139,M140,M141,M142,M143,M144,M145,M146,M147,M148,M149
1,1,sample_1,QC,QC,90.1,491.6,202.9,35.0,164.2,19.7,41.0,46.5,17.3,106.8,61.7,75.3,79.7,35.3,28.8,245.9,5.8,122.2,90.9,47.0,49.0,37.2,422.0,155.1,21.4,16.5,NaN,19.9,65.0,22.0,18.9,97.8,274.1,26.3,110.8,13.0,...,156.8,6.6,201.7,84.6,70.2,291.2,32.9,112.3,198.7,128.5,109.4,3.7,18.5,266.1,431.2,145.1,35.7,NaN,48.9,2445.7,37.9,1258.4,168.5,32.7,706.9,1106.9,107.6,2807.8,481.0,68.4,115.1,64.8,25.5,473.9,26.5,NaN,6.8,118.6,710.6,203.6
2,2,sample_2,Sample,GC,43.0,525.7,130.2,NaN,694.5,114.5,37.9,125.7,57.8,NaN,490.6,203.4,330.8,NaN,210.7,150.3,71.5,553.1,217.7,207.9,238.8,297.2,591.2,446.4,28.3,47.9,126.8,12.9,291.9,58.5,1336.6,621.2,776.7,324.5,282.8,154.3,...,69.3,150.6,477.6,3.3,217.9,66.6,131.0,694.5,635.3,209.7,237.9,47.4,41.8,188.3,673.1,283.1,61.7,367.0,225.9,961.8,100.1,1636.7,206.6,148.5,6674.1,9079.2,NaN,938.9,6084.5,75.3,84.2,357.1,16.1,455.5,29.5,28.1,35.8,316.1,390.7,199.0
3,3,sample_3,Sample,BN,214.3,10703.2,104.7,46.8,483.4,152.3,110.1,85.1,238.3,48.0,2441.2,100.0,873.3,29.3,45.4,226.4,36.3,371.9,98.1,116.5,30.3,24.6,593.4,232.6,35.1,26.8,78.4,3772.3,144.3,52.8,0.2,360.1,532.3,507.0,3207.5,161.7,...,83.2,2664.3,550.5,1319.4,413.9,55.5,42.5,25.7,271.9,88.9,105.4,40.4,39.0,67.6,997.8,180.6,76.2,145.4,494.4,673.4,42.8,1954.2,193.6,129.8,787.2,1404.5,1564.3,1163.2,246.0,2460.4,993.5,1698.5,32.9,75.9,33.2,802.8,967.6,154.4,31.6,195.2
4,4,sample_4,Sample,HE,31.6,59.7,86.4,14.0,88.6,10.3,170.3,23.9,NaN,NaN,140.7,12.6,46.3,62.9,38.3,49.7,0.4,31.3,38.6,53.2,45.4,3.7,130.1,1.6,26.6,10.3,22.8,14.5,25.5,11.2,4.8,111.6,133.4,37.3,NaN,34.5,...,31.6,22.4,18.8,90.7,39.6,136.2,11.7,91.1,59.1,16.1,NaN,NaN,12.3,50.1,184.1,102.1,31.1,0.7,649.8,651.7,31.3,799.8,34.0,22.1,392.4,550.0,43.4,370.6,109.3,68.7,58.1,83.5,60.5,136.9,17.0,10.2,24.7,64.1,91.4,91.6
5,5,sample_5,Sample,GC,81.9,258.7,315.1,8.7,243.2,18.4,349.4,61.1,12.2,72.9,48.7,57.3,140.1,77.8,51.0,71.3,18.8,265.0,84.7,78.8,141.3,58.0,722.1,174.7,58.9,65.9,2.8,15.3,63.5,38.1,11.2,233.6,328.4,79.4,7.1,305.9,...,47.5,78.2,538.8,133.3,65.1,411.1,48.9,18.8,225.8,91.5,42.9,40.3,3.9,235.0,585.5,624.2,122.4,0.1,726.1,2048.9,NaN,2380.8,168.9,58.5,973.9,1494.7,85.8,984.0,1037.4,66.4,44.5,47.6,45.6,1441.7,35.2,0.1,22.8,135.0,322.3,254.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,sample_136,QC,QC,97.6,341.1,232.1,38.1,174.0,7.7,38.4,46.3,36.8,88.4,80.9,49.5,77.9,27.9,30.7,284.0,9.1,84.4,72.4,26.7,15.7,9.0,312.4,155.2,19.0,16.0,4.6,15.5,67.4,31.8,19.0,93.1,287.3,NaN,160.2,12.0,...,102.2,7.3,205.4,127.0,82.4,300.5,30.9,144.4,195.9,179.1,80.7,4.1,17.5,220.1,496.7,138.0,36.9,33.6,NaN,2274.1,41.0,1229.3,182.4,53.7,723.6,1134.4,98.4,2792.5,451.8,1.0,79.7,101.8,23.9,296.0,25.0,NaN,7.5,141.5,675.7,200.8
137,137,sample_137,Sample,GC,405.3,510.7,521.9,91.9,732.1,145.7,492.6,95.1,143.2,150.1,194.7,124.2,686.4,166.7,79.4,277.5,187.6,375.1,192.4,255.6,11.0,65.7,847.3,221.9,171.8,132.5,112.5,129.6,266.7,783.5,156.9,225.8,1070.4,137.7,NaN,143.6,...,81.4,251.6,667.2,2.0,323.5,2134.5,41.6,1301.8,137.1,218.9,140.8,7.8,158.6,1418.2,742.5,768.7,609.4,139.2,987.3,3521.4,96.8,3776.7,292.8,205.9,2731.0,5342.2,250.1,683.1,3241.9,304.0,434.8,84.8,182.3,110.7,123.9,0.4,36.3,60.1,317.3,401.7
138,138,sample_138,Sample,BN,45.4,191.6,41.0,18.7,40.8,32.2,46.5,25.9,7.3,42.2,50.7,30.5,500.0,26.3,21.0,13.2,12.6,44.8,23.7,12.7,NaN,12.7,84.5,0.8,13.6,6.8,NaN,27.9,690.7,134.1,8.8,

- M1,..., M49 : metabolite concentrations

- SampleType: whether the sample was pooled QC or a study sample

- Class: GC = Gastric Cancer, BN = Benign Tumor, HE = Healthy Control

In [10]:
display(peakTable)

,Idx,Name,Label,Perc_missing,QC_RSD
1,1,M1,1_3-Dimethylurate,11.428571,32.208005
2,2,M2,1_6-Anhydro-β-D-glucose,0.714286,31.178028
3,3,M3,1_7-Dimethylxanthine,5.000000,34.990605
4,4,M4,1-Methylnicotinamide,8.571429,12.804201
5,5,M5,2-Aminoadipate,1.428571,9.372664
...,...,...,...,...,...
145,145,M145,uarm1,23.571429,41.406985
146,146,M146,uarm2,4.285714,34.458172
147,147,M147,β-Alanine,1.428571,27.623517
148,148,M148,π-Methylhistidine,1.428571,16.561921


- Idx: unique metabolite index

- Name: column header corresp. to this metabolite in the `dataTable` table

- Label: unique name for metabolite (or a `uNN` identifier)

- Perc_missing: what % of samples do not contain a measurement for this metabolite (missing data)

- QC_RSD: quality score representing the variation in measurements of this metabolite across sample

# 3. Data Cleaning

Keep only metabolites that meet the following criteria:

- a QC-RSD < 20%

- < 10% values are missing

In [12]:
# Create a clean peak table
rsd = peakTable['QC_RSD']
percMiss = peakTable['Perc_missing']
peakTableClean = peakTable[(rsd < 20) & (percMiss < 10)]

print('Number of peaks remaining: {}'.format(len(peakTableClean)))

Number of peaks remaining: 52


# 4. PCA - Quality Assessment

- multivariate assessment of the cleaned data

- PCA score plot: typically labelled by sample type (i.e. QC & biological sample). Data of high quality will have QCs that cluster tightly compared to the biological samples.

First the metabolite data matrix is extracted from the dataTable, and transformed & scaled:
- A new variable peaklist is created, to hold the names (M1...Mn) of the metabolites to be used in subsequent statistical analysis

- The peak data for all samples, corresponding to this list, is extracted from the dataTable table, and placed in a matrix X

- The values in X are log-transformed (Xlog)

- The helper function cb.utils.scale() is used to scale the log-transformed data (Xscale)

- Missing values are imputed using a k-nearest neighbour approach (with three neighbours) to give the table Xknn

- The transformed & scaled dataset Xknn is used as input to PCA, using the helper function cb.plot.pca(). This returns plots of PCA scores and PCA loadings, for interpretation and quality assessment.

In [16]:
# Extract and scale the metabolite data from the dataTable

peaklist = peakTableClean['Name']     # set peak list to the metabolite names in the peakTableClean
X = dataTable[peaklist].values
Xlog = np.log10(X)
Xscale = cb.utils.scale(Xlog, method='auto')    # methods: auto, pareto, vast, level
Xknn = cb.utils.knnimpute(Xscale, k=3)

print('Xknn: {} rows & {} columns'.format(*Xknn.shape))

cb.plot.pca(Xknn,
            pcx=1,
            pcy=2,
            group_label=dataTable['SampleType'])

Xknn: 140 rows & 52 columns


# 5. Univariate Statistics for comparison of Gastric Cancer (GC) vs Healthy Controls (HE)

^1H-NMR urine metabolite profiles of individuals classified into 3 distinct groups: GC (Gastric Cancer), BN (Benign), and HE (healthy). 

The helper function cb.utils.univariate_2class() will take as input a data table where the observations represent data from two groups, and a corresponding table of metabolite peak information, and produce as output summary statistics of univariate comparisons between the two groups. The output is returned as a pandas dataframe, describing output from statistical tests such as Student's t-test and Shapiro-Wilks, and summaries of data quality, like the number and percentage of missing values.

First, we reduce the data in dataTable to only those observations for GC and HE samples, and we define the GC class to be a positive outcome, in the variable pos_outcome. Next, we pass the reduced dataset and the cleaned peakTable to cb.utils.univariate_2class(), and store the returned dataframe in a new variable called statsTable. This is then displayed as before for interactive inspection and interpretation.

In [17]:
# Select subset of Data for statistical comparison
dataTable2 = dataTable[(dataTable.Class == 'GC') | (dataTable.Class == 'HE')] # Reduce data table only to GC and HE class members
pos_outcome = 'GC'

# Calculate basic stats and create a stats table
statsTable = cb.utils.univariate_2class(dataTable2,
                                        peakTableClean,
                                        group='Class',
                                        posclass=pos_outcome,
                                        parametric=True)

display(statsTable)

,Idx,Name,Label,Grp0_Mean,Grp0_Mean-95CI,Grp1_Mean,Grp1_Mean-95CI,Sign,TTestStat,TTestPvalue,bhQvalue,TotalMissing,PercTotalMissing,Grp0_Missing,Grp1_Missing,ShapiroW,ShapiroPvalue,LeveneW,LevenePvalue
1,4,M4,1-Methylnicotinamide,51.739474,"(39.35, 64.13)",26.477778,"(19.98, 32.98)",0,3.482846,0.000848,0.008816,9,10.843,5.0,16.279,0.861608,9.126937e-07,9.835944,0.002478
2,5,M5,2-Aminoadipate,169.915000,"(115.14, 224.69)",265.118605,"(146.65, 383.59)",1,-1.395129,0.166791,0.420837,0,0.000,0.0,0.000,0.551547,1.591575e-14,1.714528,0.194101
3,7,M7,2-Furoylglycine,53.987179,"(31.17, 76.81)",118.525581,"(78.5, 158.55)",1,-2.672784,0.009114,0.059243,1,1.205,2.5,0.000,0.696827,1.009690e-11,5.909098,0.017299
4,8,M8,2-Hydroxyisobutyrate,79.267500,"(59.69, 98.85)",54.395349,"(42.53, 66.26)",0,2.163519,0.033448,0.158119,0,0.000,0.0,0.000,0.817766,9.915664e-09,3.216053,0.076652
5,11,M11,3-Aminoisobutyrate,171.279487,"(104.01, 238.55)",201.343902,"(107.59, 295.1)",0,-0.506244,0.614113,0.760330,3,3.614,2.5,4.651,0.629707,6.749377e-13,0.241249,0.624685
6,14,M14,3-Hydroxyisobutyrate,83.902500,"(58.8, 109.01)",61.531707,"(45.75, 77.31)",0,1.486528,0.141120,0.407680,2,2.410,0.0,4.651,0.730224,6.725048e-11,1.852628,0.177348
7,15,M15,3-Hydroxyisovalerate,62.300000,"(48.06, 76.54)",58.472093,"(44.97, 71.98)",0,0.382551,0.703054,0.812418,0,0.000,0.0,0.000,0.820793,1.225706e-08,0.110564,0.740362
8,25,M25,6-Hydroxynicotinate,20.640000,"(15.86, 25.42)",30.293023,"(21.28, 39.31)",1,-1.814600,0.073288,0.238185,0,0.000,0.0,0.000,0.735144,6.198545e-11,1.828066,0.180119
9,26,M26,ATP,22.813514,"(17.62, 28.0)",39.944737,"(20.29, 59.6)",1,-1.632723,0.106834,0.326785,8,9.639,7.5,11.628,0.455527,3.358144e-15,2.474440,0.120035
10,31,M31,Adipate,57.615000,"(31.6, 83.63)",80.041860,"(18.98, 141.1)",0,-0.645264,0.520580,0.694410,0,0.000,0.0,0.000,0.359728,2.820873e-17,0.466944,0.496346


In [18]:
# Save StatsTable to Excel
statsTable.to_excel("stats.xlsx", sheet_name='StatsTable', index=False)
print("done!")

done!


# 6. Machine Learning

- 2-class Partial Least Square Discriminant Analysis (PLS-DA) to identify metabolites which, when combined in linear equation, are able to classify unknown samples as either GC and HE w/ a measurable degree of certainty

1. Train and Test sets

Multivariate predictive models are prone to overfitting. In order to provide some level of independent evaluation it is common practice to split the source data set into two parts: training set and test set. The model is then optimised using the training data and independently evaluated using the test data. The true effectiveness of a model can only be assessed using the test data (Westerhuis et al. 2008, Xia et al. 2012). It is vitally important that both the training and test data are equally representative of the the sample population (in our example the urine metabotype of Gastric Cancer and the urine metabotype of Healthy Control). It is typical to split the data using a ratio of 2:1 (⅔ training, ⅓ test) using stratified random selection. If the purpose of model-building is exploratory, or sample numbers are small, this step is often ignored; however, care must be taken in interpreting a model that has not been tested on a dataset that is independent of the data it was trained on.
We use the dataTable2 dataframe created above, which contains a subset of the complete data suitable for a 2-class comparision (GC vs HE). Our goal is to split this dataframe into a training subset (dataTrain) which will be used to train our model, and a test set (dataTest), which will be used to evaluate the trained model. We will split the data such that number of test set samples is 25% of the the total. To do this, we will use the scikit-learn module's train_test_split() function.

First, we need to ensure that the sample split - though random - is stratified so that the class membership is balanced to the same proportions in both the test and training sets. In order to do this, we need to supply a binary vector indicating stratification group membership.

The train_test_split() function expects a binary (1/0) list of positive/negative outcome indicators, not the GC/HE classes that we have. We convert the class information for each sample in dataTable2 into Y, a list of 1/0 values, in the code cell below.

In [25]:
# Create a Binary Y vector for stratifiying the samples
outcomes = dataTable2['Class']                                  # Column that corresponds to Y class (should be 2 groups)
Y = [1 if outcome == 'GC' else 0 for outcome in outcomes]       # Change Y into binary (GC = 1, HE = 0)  
Y = np.array(Y)                                                 # convert boolean list into to a numpy array

In [26]:
dataTrain, dataTest, Ytrain, Ytest = train_test_split(dataTable2, Y, test_size=0.25, stratify=Y,random_state=10)

print("DataTrain = {} samples with {} positive cases.".format(len(Ytrain),sum(Ytrain)))
print("DataTest = {} samples with {} positive cases.".format(len(Ytest),sum(Ytest)))

DataTrain = 62 samples with 32 positive cases.
DataTest = 21 samples with 11 positive cases.


2. Determine optimal # of components for PLS-DA model

- k-fold cross validation

- Coefficient of determination $R^2$ and cross-validated coefficient of determination $Q^2$.

if values for $R^2$ and $Q^2$ are plotted against model complexity (# of latent variables), typically the value of $Q^2$ will be seen to rise and then fall. The point at which $Q^2$ value begins to diverge from the $R^2$ value is considered the point at which the optimal # of components has been met without overfitting

In [27]:
# Extract and scale the metabolite data from the dataTable
peaklist = peakTableClean['Name']                           # Set peaklist to the metabolite names in the peakTableClean
XT = dataTrain[peaklist]                                    # Extract X matrix from DataTrain using peaklist
XTlog = np.log(XT)                                          # Log scale (base-10)
XTscale = cb.utils.scale(XTlog, method='auto')              # methods include auto, pareto, vast, and level
XTknn = cb.utils.knnimpute(XTscale, k=3)                    # missing value imputation (knn - 3 nearest neighbors)

In [28]:
# initalise cross_val kfold (stratified) 
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                   # model; we are using the PLS_SIMPLS model
                        X=XTknn,                                 
                        Y=Ytrain,                               
                        param_dict={'n_components': [1,2,3,4,5,6]},  # The numbers of latent variables to search                
                        folds=5,                                     # folds; for the number of splits (k-fold)
                        bootnum=100)                                 # num bootstraps for the Confidence Intervals

# run the cross validation
cv.run()  

Kfold: 100%|██████████| 100/100 [00:04<00:00, 21.47it/s]


In [29]:
cv.plot()

3. Train and evaluate PLS-DA model

In [30]:
modelPLS = cb.model.PLS_SIMPLS(n_components=2)

In [31]:
Ypred = modelPLS.train(XTknn, Ytrain)

In [32]:
modelPLS.evaluate(cutoffscore=0.5)

4. Perform a permutation test for the PLS-DA model

The original data is randomised/permuted so that the predictor variables & response vairables are mixed, and a new model is then trained and tested on the shuffled data. This is repeated many times so that the behaviour of models constructed from 'random' data can be fairly assessed.

Confident that our model is being trained on relevant and meaningful features of the original dataset if $R^2$ and $Q^2$ values generated from these models (w/ randomised data) are much lower than those found for our model trained on the original data.


In [33]:
modelPLS.permutation_test(nperm=100)

Permutation Resample: 100%|██████████| 100/100 [00:01<00:00, 77.39it/s]


5. Plot latent variable projections for PLS-DA model

- diagonal shows pdf of each latent variable (LV) for each response class. LV1 is at the top left of the plot

- upper triangle shows ROC curves for each optimal discriminating pairwise combination of LVx and LVy scores

- lower triangle shows scatterplots of the scores for LVy and LVx, w/ a solid line indicating the direction of max discrimination

In [34]:
modelPLS.plot_projections(label=dataTrain[['Idx', 'SampleID']], size=12)

6. Plot feature importance (coefficient plot and VIP) for PLS-DA model

- Determine the importance of specific peaks to the model's discriminatory power

- PLS regression coefficient values for each metabolite and Variable Important in Projection (VIP) plots

- coeff values: info about the contribution of the peak to either a -ve/+ve classification for the sample

- peaks w/ VIP > unity (1) are considered to be 'important' in the model

In [35]:
# Calculate the bootstrapped CI
modelPLS.calc_bootci(type='bca', bootnum=200)

# Plot the feature importance plots, and return a new Peaksheet
peakSheet = modelPLS.plot_featureimportance(peakTableClean, 
                                            peaklist,
                                            ylabel='Label',
                                            sort=False)

Jackknife Resample: 100%|██████████| 62/62 [00:00<00:00, 1048.32it/s]


7. Test model w/ new data (using test set)

In [36]:
mu, sigma = cb.utils.scale(XTlog, return_mu_sigma=True)

In [37]:
# Pull of Xtest from DataTest using peaklist ('Name' column in PeakTable)
peaklist = peakTableClean.Name 
XV = dataTest[peaklist].values

# Log transform, unit-scale and knn-impute missing values for Xtest
XVlog = np.log(XV)
XVscale  = cb.utils.scale(XVlog, method='auto', mu=mu, sigma=sigma) 
XVknn = cb.utils.knnimpute(XVscale, k=3)

In [38]:
YVpred = modelPLS.test(XVknn)

evals = [Ytest, YVpred]

modelPLS.evaluate(evals, cutoffscore=0.5)

In [39]:
# Save DataSheet as 'Idx', 'SampleID', and 'Class' from DataTest
dataSheet = dataTest[["Idx", "SampleID", "Class"]].copy() 

# Add 'Ypred' to Datasheet
dataSheet['Ypred'] = YVpred 
 
display(dataSheet) # View and check the dataTable 


,Idx,SampleID,Class,Ypred
4,4,sample_4,HE,0.596320
78,78,sample_78,GC,0.762340
90,90,sample_90,HE,0.190719
71,71,sample_71,GC,0.799896
92,92,sample_92,GC,1.046995
119,119,sample_119,HE,0.122116
56,56,sample_56,HE,0.166406
104,104,sample_104,HE,-0.130498
98,98,sample_98,GC,0.230983
36,36,sample_36,GC,0.401337


In [40]:
# Create an empty excel workbook
writer = pd.ExcelWriter("modelPLS.xlsx")     # provide the filename for the Excel file

# Add each dataframe to the workbook in turn, as a separate worksheet
dataSheet.to_excel(writer, sheet_name='Datasheet', index=False)
peakSheet.to_excel(writer, sheet_name='Peaksheet', index=False)

# Write the Excel workbook to disk
writer.save()

print("Done!")

Done!
